# Prédire la demande d’un article donné

## Objectif

Réduire le nombre d'articles unique sans perdre trop de lignes du dataset

1. Regroupe certains articles sous la même appelation (ex : galette 4P et galette 6P sous le nom "galette")
2. Supprimer les lignes des articles très peu vendus (représentant <SEUIL % des ventes totales)

---

## Résultats de la 1ere idée de réduction (CE N'EST PLUS ALIGNÉ AVEC LE PROJET "PREDIRE QTÉ INGRÉDIENT DES TOP ARTICLE")

Nombre d'article unique avant modification = 149

Nombre d'article unique après Regroupement = 110

Nombre d'article unique après Suppression = 60

Nombre de lignes avant modifications = 234005

Nombre de lignes après suppression des articles très peu vendus = 228107



# 1. Import du dataset original 

In [181]:
# importer le dataset
import pandas as pd
data = pd.read_csv('data_tmp/bakery_sales.csv')
data.head()

print(f"Nombre de lignes avant modifications = {len(data)}")
print(f"Nombre d'article unique avant Suppression = {data['article'].nunique()}")

Nombre de lignes avant modifications = 234005
Nombre d'article unique avant Suppression = 149


# 2. Exploration et nettoyage  du dataset. 

## Objectif

1. Vérifier quels articles ont été très peu vendus (peu d’occurrences dans le dataset).  
2. Afficher la liste complète des articles uniques, triés par ordre alphabétique.

## A. Vérification des colonnes disponibles et des valeurs manquantes
CONCLUSION : Rien de manquant

In [182]:
# Vérifier les colonnes et types de données
print(data.info())

# Vérifier les valeurs manquantes
# AUCUNE

# Vérifier s'il y a des doublons exacts
nb_duplicated = data.duplicated().sum()
print(f"Nombre de lignes dupliquées : {nb_duplicated}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234005 entries, 0 to 234004
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     234005 non-null  int64  
 1   date           234005 non-null  object 
 2   time           234005 non-null  object 
 3   ticket_number  234005 non-null  float64
 4   article        234005 non-null  object 
 5   Quantity       234005 non-null  float64
 6   unit_price     234005 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 12.5+ MB
None
Nombre de lignes dupliquées : 0


## A.bis Correction des quantités négatives

Il y avait certaines lignes avec des quantités négatives. Après études ça corrrespond à des articles qui ont été remboursés. Nous supprimons donc les lignes négatives et les lignes des achats correspondants

In [183]:
# Nombre de lignes avant modifications
print(f"Nombre de lignes avant modifications = {len(data)}")

### On commence par récuperer les ticket_number de tous les tickets négatifs
remb_tickets = data.loc[data['Quantity'] < 0, 'ticket_number']

### On récupère les ticket_number des tickets qu'ils remboursent et on les join dans un set
tickets_a_supprimer = set(remb_tickets).union(remb_tickets - 1)

### Enfin on supprimme de la BDD tous les tickets number des tickets à rembourser et les tickets remboursés
data = data[~data['ticket_number'].isin(tickets_a_supprimer)]

# Nombre de lignes après modifications
print(f"Nombre de lignes après modifications = {len(data)}")

Nombre de lignes avant modifications = 234005
Nombre de lignes après modifications = 231423


## B. Compter le nombre de ventes par article, pour supprimer quelques valeurs "Article" inutiles

CONCLUSION : 

### Afficher la liste des noms d'articles et leur vente total pour faire la selection des modifications

In [184]:
# Nombre total de lignes par article
ventes_par_article = data["article"].value_counts()
nombre_articles_uniques = len(ventes_par_article)
print(f"Nombre d'articles uniques avant modification : {nombre_articles_uniques}")

#Trier les articles par ordre alphabétique
ventes_par_article = ventes_par_article.sort_index()

#Afficher tous les articles avec leur nombre de ventes
print(ventes_par_article.to_string())

Nombre d'articles uniques avant modification : 148
article
.                               5
12 MACARON                     65
ARMORICAIN                      2
ARTICLE 295                     1
BAGUETTE                    15161
BAGUETTE APERO                 59
BAGUETTE GRAINE              1489
BANETTE                     14909
BANETTINE                    2795
BOISSON 33CL                 1451
BOTTEREAU                      85
BOULE 200G                   2667
BOULE 400G                   4049
BOULE POLKA                   498
BRIOCHE                      1642
BRIOCHE DE NOEL                19
BRIOCHETTE                     46
BROWNIES                       36
BUCHE 4PERS                    10
BUCHE 6PERS                     8
BUCHE 8PERS                     1
CAFE OU EAU                  1419
CAKE                            1
CAMPAGNE                     3885
CARAMEL NOIX                   69
CEREAL BAGUETTE              4906
CHAUSSON AUX POMMES          1434
CHOCOLAT               

### Regroupement proposés
BAGUETTE APERO                 73 --> BAGUETTE
BRIOCHE DE NOEL                19 --> BRIOCHE
BRIOCHETTE                     46 --> BRIOCHE
DEMI PAIN                     163 --> DEMI PAIN
ECLAIR FRAISE PISTACHE         16 --> ECLAIR
FINANCIER                      17 --> FINANCIER X5
GD FAR BRETON                 161 --> GRAND FAR BRETON
PAIN GRAINES                    3 --> PAIN
PAIN NOIR                       1 --> PAIN
PAIN S/SEL                    131 --> PAIN
PAIN SUISSE PEPITO             18 --> PAIN
ROYAL 4P                       61 --> ROYAL
ROYAL 6P                       62 --> ROYAL
SACHET DE VIENNOISERIE          1 --> SACHET VIENNOISERIE
SAND JB                         5 --> SAND JB EMMENTAL
SPECIAL BREAD KG              460 --> SPECIAL BREAD
TARTELETTE CHOC                52 --> TARTELETTE
TARTELETTE COCKTAIL             1 --> TARTELETTE
TARTELETTE FRAISE             493 --> TARTELETTE
TROPEZIENNE FRAMBOISE          26 --> TROPEZIENNE


### Création (merger des articles différents dans un nouvel article)
GAL FRANGIPANE 4P             203 --> GALETTE FRANGIPANE 
GAL FRANGIPANE 6P             122 --> GALETTE FRANGIPANE

GAL POIRE CHOCO 4P             24 --> GALETTE FRUIT
GAL POIRE CHOCO 6P             15 --> GALETTE FRUIT
GAL POMME 4P                   97 --> GALETTE FRUIT
GAL POMME 6P                   61 --> GALETTE FRUIT
GALETTE 8 PERS                  2 --> GALETTE FRUIT

PLAT                            8 --> PLAT PREPARE
PLAT 6.50E                      2 --> PLAT PREPARE
PLAT 7.00                      94 --> PLAT PREPARE
PLAT 7.60E                    244 --> PLAT PREPARE
PLAT 8.30E                    116 --> PLAT PREPARE
PLATPREPARE5,50                 3 --> PLAT PREPARE
PLATPREPARE6,00                 1 --> PLAT PREPARE
PLATPREPARE6,50                 6 --> PLAT PREPARE
PLATPREPARE7,00                11 --> PLAT PREPARE

TARTE FINE                     61 --> TARTE FRUITS
TARTE FRAISE 4PER             149 --> TARTE FRUITS
TARTE FRAISE 6P               125 --> TARTE FRUITS
TARTE FRUITS 4P               358 --> TARTE FRUITS
TARTE FRUITS 6P               272 --> TARTE FRUITS

### On trie les articles du moins vendu au plus vendu, puis on prend les premiers tant que la somme cumulée reste ≤ 1 % des ventes totales.

In [185]:

# Comptage des ventes par article (somme des Quantities)
ventes_par_article = data.groupby("article")["Quantity"].sum()

# Connaitre les articles dont le cumul représente au plus 1% des ventes totales
ventes_totales = ventes_par_article.sum()
seuil_1pct_ventes = ventes_totales * 0.01

print(f"Seuil pour 1% des ventes totales : {seuil_1pct_ventes:.0f} ventes")

# Trier les articles du moins vendu au plus vendu (par unités vendues)
ventes_tries = ventes_par_article.sort_values(ascending=True)

# Somme cumulée des ventes en partant du moins vendu
cumul_ventes = ventes_tries.cumsum()

# Garder les articles tant que le cumul reste <= 1% des ventes totales
articles_sous_1pct = ventes_tries[cumul_ventes <= seuil_1pct_ventes]

print("Articles les moins vendus dont les ventes cumulées représentent au plus 1% des ventes totales :")
print(articles_sous_1pct.to_string())

print(f"\nTotal cumulé de ces articles : {articles_sous_1pct.sum()} ventes")
print(f"Ce qui représente {articles_sous_1pct.sum() / ventes_totales * 100:.3f}% des ventes totales.")

Seuil pour 1% des ventes totales : 3600 ventes
Articles les moins vendus dont les ventes cumulées représentent au plus 1% des ventes totales :
article
SACHET DE VIENNOISERIE        1.0
PAIN NOIR                     1.0
REDUCTION SUCREES 24          1.0
ARTICLE 295                   1.0
CAKE                          1.0
DOUCEUR D HIVER               1.0
TROIS CHOCOLAT                1.0
PLATPREPARE6,00               1.0
BUCHE 8PERS                   2.0
FORMULE PATE                  2.0
GD PLATEAU SALE               2.0
GALETTE 8 PERS                2.0
TARTELETTE COCKTAIL           2.0
PATES                         2.0
TULIPE                        2.0
PLAQUE TARTE 25P              2.0
PLATPREPARE5,50               3.0
GUERANDAIS                    3.0
PAIN GRAINES                  3.0
ARMORICAIN                    3.0
CRUMBLECARAMEL OU PISTAE      3.0
PT PLATEAU SALE               4.0
SAND JB                       5.0
PALMIER                       6.0
THE                           7.0

# 3. Reduction du dataset (LE SEUIL DES TOP ARTICLE EST FIXÉ ICI)

Objectif : Ne garder que les lignes à propos de TOP ARTICLES (i.e. ceux qui represente + de 75-85% des ventes totales)

## A. Regroupement de certain nom d'articles

### V1 CODE MORT (mais problème de variation des prix entre articles)



In [186]:
'''
# 1) Normaliser (majuscule + strip)
data["article_norm"] = data["article"].astype(str).str.upper().str.strip()

# 2) Dictionnaire mapping exact (source -> cible) basé sur ta liste
mapping_exact = {
    # Regroupement proposés
    "BRIOCHE DE NOEL": "BRIOCHE",
    "BRIOCHETTE": "BRIOCHE",
    "DEMI PAIN": "DEMI PAIN",
    "ECLAIR FRAISE PISTACHE": "ECLAIR",
    "FINANCIER": "FINANCIER X5",
    "GD FAR BRETON": "GRAND FAR BRETON",
    "GD KOUIGN AMANN": "KOUIGN AMANN",
    "GD NANTAIS": "NANTAIS",
    "PT NANTAIS": "NANTAIS",
    "GRANDE SUCETTE": "SUCETTE",
    "PAIN GRAINES": "PAIN",
    "PAIN NOIR": "PAIN",
    "PAIN S/SEL": "PAIN",
    "PAIN SUISSE PEPITO": "PAIN",
    "ROYAL 4P": "ROYAL",
    "ROYAL 6P": "ROYAL",
    "SACHET VIENNOISERIE": "SACHET DE VIENNOISERIE",
    "SAND JB": "SAND JB EMMENTAL",
    "SPECIAL BREAD KG": "SPECIAL BREAD",
    "TARTELETTE CHOC": "TARTELETTE",
    "TARTELETTE COCKTAIL": "TARTELETTE",
    "TARTELETTE FRAISE": "TARTELETTE",
    "TROPEZIENNE FRAMBOISE": "TROPEZIENNE",
    # Création / mergers (on met aussi en exact pour la sécurité)
    "BOULE 200G": "BOULE GLACE",
    "BOULE 400G": "BOULE GLACE",
    "BUCHE 4PERS": "BUCHE",
    "BUCHE 6PERS": "BUCHE",
    "BUCHE 8PERS": "BUCHE",
    "GAL FRANGIPANE 4P": "GALETTE FRANGIPANE",
    "GAL FRANGIPANE 6P": "GALETTE FRANGIPANE",
    "GAL POIRE CHOCO 4P": "GALETTE FRUIT",
    "GAL POIRE CHOCO 6P": "GALETTE FRUIT",
    "GAL POMME 4P": "GALETTE FRUIT",
    "GAL POMME 6P": "GALETTE FRUIT",
    "GALETTE 8 PERS": "GALETTE FRUIT",
    "PLAT": "PLAT PREPARE",
    "PLAT 6.50E": "PLAT PREPARE",
    "PLAT 7.00": "PLAT PREPARE",
    "PLAT 7.60E": "PLAT PREPARE",
    "PLAT 8.30E": "PLAT PREPARE",
    "PLATPREPARE5,50": "PLAT PREPARE",
    "PLATPREPARE6,00": "PLAT PREPARE",
    "PLATPREPARE6,50": "PLAT PREPARE",
    "PLATPREPARE7,00": "PLAT PREPARE",
    "TARTE FINE": "TARTE FRUITS",
    "TARTE FRAISE 4PER": "TARTE FRUITS",
    "TARTE FRAISE 6P": "TARTE FRUITS",
    "TARTE FRUITS 4P": "TARTE FRUITS",
    "TARTE FRUITS 6P": "TARTE FRUITS",
}


# 4) Appliquer mapping exact
data["article_reduced"] = data["article_norm"].map(mapping_exact)



# 6) Remplir le reste par l'article normalisé (ou choisir de garder l'original si préféré)
data["article_reduced"] = data["article_reduced"].fillna(data["article_norm"])

# 7) Vérifications rapides
print("Nombre d'articles uniques avant modification :", data["article"].nunique())
print("Nombre d'articles uniques après regroupement :", data["article_reduced"].nunique())

'''

'\n# 1) Normaliser (majuscule + strip)\ndata["article_norm"] = data["article"].astype(str).str.upper().str.strip()\n\n# 2) Dictionnaire mapping exact (source -> cible) basé sur ta liste\nmapping_exact = {\n    # Regroupement proposés\n    "BRIOCHE DE NOEL": "BRIOCHE",\n    "BRIOCHETTE": "BRIOCHE",\n    "DEMI PAIN": "DEMI PAIN",\n    "ECLAIR FRAISE PISTACHE": "ECLAIR",\n    "FINANCIER": "FINANCIER X5",\n    "GD FAR BRETON": "GRAND FAR BRETON",\n    "GD KOUIGN AMANN": "KOUIGN AMANN",\n    "GD NANTAIS": "NANTAIS",\n    "PT NANTAIS": "NANTAIS",\n    "GRANDE SUCETTE": "SUCETTE",\n    "PAIN GRAINES": "PAIN",\n    "PAIN NOIR": "PAIN",\n    "PAIN S/SEL": "PAIN",\n    "PAIN SUISSE PEPITO": "PAIN",\n    "ROYAL 4P": "ROYAL",\n    "ROYAL 6P": "ROYAL",\n    "SACHET VIENNOISERIE": "SACHET DE VIENNOISERIE",\n    "SAND JB": "SAND JB EMMENTAL",\n    "SPECIAL BREAD KG": "SPECIAL BREAD",\n    "TARTELETTE CHOC": "TARTELETTE",\n    "TARTELETTE COCKTAIL": "TARTELETTE",\n    "TARTELETTE FRAISE": "TARTELETTE"

In [187]:
# créer un csv avec la liste des articles uniques dans l'ordre alphabétique
# unique_articles_initial = data["article"].unique()
# unique_articles_initial.sort()
# pd.DataFrame(unique_articles_initial, columns=["article"]).to_csv("data_tmp/unique_articles_initial.csv", index=False)

# unique_articles = data["article_reduced"].unique()
# unique_articles.sort()
# pd.DataFrame(unique_articles, columns=["article_reduced"]).to_csv("data_tmp/unique_articles_reduced1.csv", index=False)

In [188]:
'''
# Remettre la colonne article à la place de article_reduced et supprimer les colonnes intermédiaires
data["article"] = data["article_reduced"]
data = data.drop(columns=["article_norm", "article_reduced"])
'''

'\n# Remettre la colonne article à la place de article_reduced et supprimer les colonnes intermédiaires\ndata["article"] = data["article_reduced"]\ndata = data.drop(columns=["article_norm", "article_reduced"])\n'

### V2 Avec seulement des articles à prix constants (on mettra la prix de l'article le + représenté )

In [189]:
# 1) Normaliser (majuscule + strip)
data["article_norm"] = data["article"].astype(str).str.upper().str.strip()

# 2) Dictionnaire mapping exact (source -> cible) basé sur ta liste
mapping_exact = {
    # Regroupement proposés
    "FINANCIER X5": "FINANCIER",
    "GRANDE SUCETTE": "SUCETTE",
    "SACHET VIENNOISERIE": "SACHET DE VIENNOISERIE",
    "SAND JB EMMENTAL": "SAND JB",
    # Création / mergers (on met aussi en exact pour la sécurité)
    "BUCHE 4PERS": "BUCHE",
    "BUCHE 6PERS": "BUCHE",
    "BUCHE 8PERS": "BUCHE",
    "GAL FRANGIPANE 4P": "GALETTE FRANGIPANE",
    "GAL FRANGIPANE 6P": "GALETTE FRANGIPANE",
    "GAL POIRE CHOCO 4P": "GALETTE FRUIT",
    "GAL POIRE CHOCO 6P": "GALETTE FRUIT",
    "GAL POMME 4P": "GALETTE FRUIT",
    "GAL POMME 6P": "GALETTE FRUIT",
    "GALETTE 8 PERS": "GALETTE FRUIT",
    "PLAT 6.50E": "PLAT PREPARE",
    "PLAT 7.00": "PLAT PREPARE",
    "PLAT 7.60E": "PLAT PREPARE",
    "PLAT 8.30E": "PLAT PREPARE",
    "PLATPREPARE5,50": "PLAT PREPARE",
    "PLATPREPARE6,00": "PLAT PREPARE",
    "PLATPREPARE6,50": "PLAT PREPARE",
    "PLATPREPARE7,00": "PLAT PREPARE",
}


# 4) Appliquer mapping exact
data["article_reduced"] = data["article_norm"].map(mapping_exact)



# 6) Remplir le reste par l'article normalisé (ou choisir de garder l'original si préféré)
data["article_reduced"] = data["article_reduced"].fillna(data["article_norm"])

# 7) Vérifications rapides
print("Nombre d'articles uniques avant modification :", data["article"].nunique())
print("Nombre d'articles uniques après regroupement :", data["article_reduced"].nunique())

# Remettre la colonne article à la place de article_reduced et supprimer les colonnes intermédiaires
data["article"] = data["article_reduced"]
data = data.drop(columns=["article_norm", "article_reduced"])

# MODIFICATION PRIX
# créer une colonne numérique pour calculs (ex: "0,90 €" -> 0.90)
data['unit_price_num'] = (
    data['unit_price'].astype(str)
        .str.replace(r'[^0-9,.-]', '', regex=True)   # garde chiffres, virgule, point, -
        .str.replace(',', '.', regex=False)          # virgule -> point
)
data['unit_price_num'] = pd.to_numeric(data['unit_price_num'], errors='coerce')

# 1) Unifier par la moyenne pour certains articles (format string "0,90 €")
to_average = ['BUCHE', 'GALETTE FRANGIPANE', 'GALETTE FRUIT', 'PLAT PREPARE','FINANCIER',
               'SUCETTE', 'SAND JB', 'SACHET DE VIENNOISERIE']

def format_euro(x):
    return f"{x:.2f}".replace('.', ',') + " €"

for art in to_average:
    vals = data.loc[data['article'] == art, 'unit_price_num'].dropna()
    if vals.empty:
        continue
    mean_val = vals.mean()
    data.loc[data['article'] == art, 'unit_price'] = format_euro(mean_val)



# supprimer colonne intermédiaire numérique
data.drop(columns=['unit_price_num'], inplace=True)




Nombre d'articles uniques avant modification : 148
Nombre d'articles uniques après regroupement : 131


## A.bis suppression de l'article "coupe"

In [190]:
#Supprimer artcle 'coupe'
data = data[data['article'] != 'COUPE']

## B. Suppression des articles dont la vente cumulée est < à SEUIL %

In [191]:

# Comptage des ventes par article (somme des Quantities)
ventes_par_article = data.groupby("article")["Quantity"].sum()

seuil = 0.25

# Connaitre les articles dont le cumul représente au plus 2% des ventes totales
ventes_totales = ventes_par_article.sum()
seuil_pct_ventes = ventes_totales * seuil 

print(f"Seuil pour {seuil*100}% des ventes totales : {seuil_pct_ventes:.0f} ventes")

# Trier les articles du moins vendu au plus vendu (par unités vendues)
ventes_tries = ventes_par_article.sort_values(ascending=True)

# Somme cumulée des ventes en partant du moins vendu
cumul_ventes = ventes_tries.cumsum()

# Garder les articles tant que le cumul reste <= seuil% des ventes totales
articles_sous_pct = ventes_tries[cumul_ventes <= seuil_pct_ventes]

print(f"Articles les moins vendus dont les ventes cumulées représentent au plus {seuil*100}% des ventes totales :")
print(articles_sous_pct.to_string())

print(f"\nTotal cumulé de ces articles : {articles_sous_pct.sum()} ventes")
print(f"Ce qui représente {articles_sous_pct.sum() / ventes_totales * 100:.3f}% des ventes totales.")

Seuil pour 25.0% des ventes totales : 84130 ventes
Articles les moins vendus dont les ventes cumulées représentent au plus 25.0% des ventes totales :
article
CAKE                           1.0
DOUCEUR D HIVER                1.0
ARTICLE 295                    1.0
TROIS CHOCOLAT                 1.0
PAIN NOIR                      1.0
REDUCTION SUCREES 24           1.0
FORMULE PATE                   2.0
GD PLATEAU SALE                2.0
PATES                          2.0
PLAQUE TARTE 25P               2.0
TARTELETTE COCKTAIL            2.0
TULIPE                         2.0
PAIN GRAINES                   3.0
ARMORICAIN                     3.0
CRUMBLECARAMEL OU PISTAE       3.0
GUERANDAIS                     3.0
PT PLATEAU SALE                4.0
PALMIER                        6.0
THE                            7.0
.                              7.0
REDUCTION SUCREES 12           8.0
PLAT                           9.0
CRUMBLE                        9.0
FORMULE PLAT PREPARE          11.0
BR

In [192]:
# suppression des lignes avec des articles très peu vendus
articles_a_supprimer = articles_sous_pct.index.tolist()
data = data[~data["article"].isin(articles_a_supprimer)]
print(f"Nombre de lignes après suppression des articles très peu vendus : {len(data)}")

print(f"Nombre d'article unique après Suppression = {data['article'].nunique()}")


Nombre de lignes après suppression des articles très peu vendus : 145771
Nombre d'article unique après Suppression = 12


La liste des 'top article' sous le seuil

In [193]:
liste_top_articles = data["article"].unique().tolist()
print("Liste des 'top articles' après suppression des articles très peu vendus :")
print(liste_top_articles)

Liste des 'top articles' après suppression des articles très peu vendus :
['BAGUETTE', 'PAIN AU CHOCOLAT', 'TRADITIONAL BAGUETTE', 'CROISSANT', 'BANETTE', 'SPECIAL BREAD', 'BOULE 400G', 'CAMPAGNE', 'CEREAL BAGUETTE', 'COOKIE', 'FORMULE SANDWICH', 'TARTELETTE']


## C. Data to csv

In [194]:
data.to_csv('data_tmp/bakery_sales_reduce.csv', index=False)

# ANNEXE CLARA (Regroupement en Famille d'articles)

In [195]:
# Families definition
boulangerie = [
    'BAGUETTE', 'PAIN', 'TRADITIONAL BAGUETTE', 'BANETTE', 'BANETTINE',
    'SPECIAL BREAD', 'COUPE', 'BOULE 200G', 'BOULE 400G', 'CAMPAGNE',
    'MOISSON', 'CEREAL BAGUETTE', 'SEIGLE', 'COMPLET', 'FICELLE',
    'VIK BREAD', 'PAIN BANETTE', 'QUIM BREAD', 'BOULE POLKA',
    'DEMI BAGUETTE', 'BAGUETTE GRAINE', 'DEMI PAIN'
]

viennoiserie = [
    'CROISSANT', 'PAIN AU CHOCOLAT', 'PAIN AUX RAISINS', 'CROISSANT AMANDES',
    'PAIN CHOCO AMANDES', 'SACHET VIENNOISERIE', 'KOUIGN AMANN',
    'GD KOUIGN AMANN', 'NANTAIS', 'BOTTEREAU', 'VIENNOISE',
    'DIVERS VIENNOISERIE'
]

patisserie = [
    'TARTELETTE', 'FLAN', 'FLAN ABRICOT', 'Paris Brest'.upper(), 'MILLES FEUILLES',
    'ECLAIR', 'CHOU CHANTILLY', 'SAVARIN', 'ROYAL', 'TARTE FRUITS',
    'TROPEZIENNE', 'FRAISIER', 'NOIX JAPONAISE', 'FONDANT CHOCOLAT',
    'GALETTE FRANGIPANE', 'GALETTE FRUIT', 'GRAND FAR BRETON'
]

confiserie = [
    'COOKIE', 'FINANCIER X5', 'SUCETTE', 'GRANDE SUCETTE', 'PALET BRETON',
    'MACARON', 'DIVERS CONFISERIE'
]

snacking = [
    'SAND JB EMMENTAL', 'SANDWICH COMPLET', 'DIVERS SANDWICHS',
    'FORMULE SANDWICH', 'BOISSON 33CL', 'CAFE OU EAU', 'TRAITEUR',
    'PLAT PREPARE'
]

# mapping
def get_famille(article):
    if article in boulangerie:
        return 'Boulangerie'
    elif article in viennoiserie:
        return 'Viennoiserie'
    elif article in patisserie:
        return 'Pâtisserie'
    elif article in confiserie:
        return 'Confiserie'
    elif article in snacking:
        return 'Snacking'
    else:
        return 'Autre'
data['Famille_traditionnelle'] = data['article'].apply(get_famille)
len(data['Famille_traditionnelle'].unique())

5